In [48]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import OPTICS
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import metrics
from sklearn.metrics import davies_bouldin_score
import umap.umap_ as umap

import pprint
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import pandas as pd
import warnings
import urllib.request
from PIL import Image
%matplotlib inline

warnings.filterwarnings("ignore")

# sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

In [49]:
top = pd.read_excel("../data/soloq/clustering/top_clustering.xlsx")
jungle = pd.read_excel("../data/soloq/clustering/jungle_clustering.xlsx")
mid = pd.read_excel("../data/soloq/clustering/mid_clustering.xlsx")
bottom = pd.read_excel("../data/soloq/clustering/bottom_clustering.xlsx")
utility = pd.read_excel("../data/soloq/clustering/utility_clustering.xlsx")

In [50]:
soloq_games_euw = pd.read_csv("../data/soloq/Europe_stats.csv")
soloq_games_kr = pd.read_csv("../data/soloq/Asia_stats.csv")

soloq_games = pd.concat([soloq_games_euw, soloq_games_kr])

In [51]:
soloq_games.head(10)

,Unnamed: 0,game_id,gameVersion,gameDuration,summonerName,puuid,participantId,teamId,teamPosition,win,...,dmg_per_minute_diff_15,gold_diff_15,xp_per_min_3_15,gold_xp_diff_15,lane_proximity,jungle_proximity,percent_mid_lane,percent_side_lanes,forward_percentage,counter_jungle_time_percentage
0,0,EUW1_5790486081,12.5.425.9171,1970,Lapin des neiges,p90mSkzSq6tzsECC-lPqUzJ1i9pcsAdcScu9ffCwCF5iBC...,1,Blue,TOP,True,...,-0.147469,-1818,20.507107,-823448,0.250000,0.750000,0.083333,0.166667,1.000000,0.750000
1,1,EUW1_5790486081,12.5.425.9171,1970,BblythewDj,Cnsnb6EpOLdn7KRc8nPud-rH3MSkBKPOO-tST1T7tlvmZg...,2,Blue,JUNGLE,True,...,0.166839,-554,15.778173,-669769,0.083333,0.916667,0.000000,0.083333,0.250000,0.250000
2,2,EUW1_5790486081,12.5.425.9171,1970,PHANTOMBLOΤ,6QobTfWdZvcIEl6t62oXW7ayFzULNQpubr5vbG6XewERxZ...,3,Blue,MIDDLE,True,...,0.323577,-6,24.145685,-801747,0.750000,0.250000,0.750000,0.000000,0.750000,0.083333
3,3,EUW1_5790486081,12.5.425.9171,1970,EL OSCUR0,Dxu3By6l_xbfkZ-SAFL9OmkMHRlvATgQxAYbCvKSH0d6BN...,4,Blue,BOTTOM,True,...,-0.331608,251,18.960406,-592227,0.250000,0.750000,0.083333,0.166667,0.083333,0.000000
4,4,EUW1_5790486081,12.5.425.9171,1970,Mark champBetter,UxgOqsDDTAfzoMsCA_38Y4j51m8AZLjmuyaHPnSUhXb0YO...,5,Blue,UTILITY,True,...,-0.287487,-94,12.915228,-499810,0.250000,0.750000,0.166667,0.083333,0.083333,0.083333
5,5,EUW1_5790486081,12.5.425.9171,1970,PapiTeero,UOaVzjc1nishktp4mxwgCHvi3l9G6HTKSbdRwVYwvLfV0w...,6,Red,TOP,False,...,0.147469,1818,26.448223,-586024,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
6,6,EUW1_5790486081,12.5.425.9171,1970,Paul227,YuVucTmrknYSLnXpz4LgAGHRY722Om3EC_vp1txUtMBl9j...,7,Red,MIDDLE,False,...,-0.323577,6,24.613706,-744624,0.666667,0.333333,0.583333,0.083333,0.416667,0.166667
7,7,EUW1_5790486081,12.5.425.9171,1970,Thomas Fäte,Mq-3FKkyj9-AP1x8ebsQYjHqThcej2XHC5oZN97RCC13gB...,8,Red,JUNGLE,False,...,-0.166839,554,17.215228,-552179,0.083333,0.916667,0.083333,0.000000,0.666667,0.666667
8,8,EUW1_5790486081,12.5.425.9171,1970,yeka1,TZfn0smhsGulOiAU-9NRHSx4XEpfcqlO4X2Y32SoNvV8IJ...,9,Red,BOTTOM,False,...,0.331608,-251,16.779695,-673407,0.583333,0.416667,0.166667,0.416667,1.000000,0.416667
9,9,EUW1_5790486081,12.5.425.9171,1970,pewpawpewpew,qyFkIjPZ1po_hv7rkpaEj-jQGy5iZ_Sa_4rio1fKcENWf-...,10,Red,UTILITY,False,...,0.287487,94,12.002030,-518378,0.083333,0.916667,0.083333,0.000000,1.000000,0.916667


In [52]:
clean_soloq_games = soloq_games[["game_id", "teamId", "teamPosition", "win", "championName", "championId"]]

In [53]:
clean_soloq_games.head(10)

,game_id,teamId,teamPosition,win,championName,championId
0,EUW1_5790486081,Blue,TOP,True,Gnar,150
1,EUW1_5790486081,Blue,JUNGLE,True,Kindred,203
2,EUW1_5790486081,Blue,MIDDLE,True,Kassadin,38
3,EUW1_5790486081,Blue,BOTTOM,True,Jhin,202
4,EUW1_5790486081,Blue,UTILITY,True,Bard,432
5,EUW1_5790486081,Red,TOP,False,Kennen,85
6,EUW1_5790486081,Red,MIDDLE,False,Akali,84
7,EUW1_5790486081,Red,JUNGLE,False,Khazix,121
8,EUW1_5790486081,Red,BOTTOM,False,Jinx,222
9,EUW1_5790486081,Red,UTILITY,False,Senna,235


In [54]:
top_soloq = clean_soloq_games[clean_soloq_games['teamPosition'] == "TOP"]
jungle_soloq = clean_soloq_games[clean_soloq_games['teamPosition'] == "JUNGLE"]
mid_soloq = clean_soloq_games[clean_soloq_games['teamPosition'] == "MIDDLE"]
bot_soloq = clean_soloq_games[clean_soloq_games['teamPosition'] == "BOTTOM"]
utility_soloq = clean_soloq_games[clean_soloq_games['teamPosition'] == "UTILITY"]

In [62]:
def return_champion(row, champions):
    for i, x in champions['championName'].items():
        if x == row['championName']:
            return champions['group'][i]
    return 999
        

In [78]:
def insert_group(clean_soloq_games, role_groups, role, group_label, winner):
    role_soloq = clean_soloq_games[clean_soloq_games['teamPosition'] == role]
    if (winner):
        role_soloq = role_soloq[role_soloq['win'] == True]
    else:
        role_soloq = role_soloq[role_soloq['win'] == False]
    role_soloq = role_soloq[role_soloq['championName'].isin(role_groups['championName'].to_list())]
    role_soloq[group_label] = role_soloq.apply(lambda row: return_champion(row, role_groups), axis=1)
    role_soloq = role_soloq[role_soloq[group_label] != 999]
    return role_soloq
    

w_top_soloq = insert_group(clean_soloq_games, top, "TOP", "w_top_group", True)
l_top_soloq = insert_group(clean_soloq_games, top, "TOP", "l_top_group", False)

w_jungle_soloq = insert_group(clean_soloq_games, jungle, "JUNGLE", "w_jungle_group", True)
l_jungle_soloq = insert_group(clean_soloq_games, jungle, "JUNGLE", "l_jungle_group", False)

w_mid_soloq = insert_group(clean_soloq_games, mid, "MIDDLE", "w_mid_group", True)
l_mid_soloq = insert_group(clean_soloq_games, mid, "MIDDLE", "l_mid_group", False)

w_bottom_soloq = insert_group(clean_soloq_games, bottom, "BOTTOM", "w_bottom_group", True)
l_bottom_soloq = insert_group(clean_soloq_games, bottom, "BOTTOM", "l_bottom_group", False)

w_utility_soloq = insert_group(clean_soloq_games, utility, "UTILITY", "w_utility_group", True)
l_utility_soloq = insert_group(clean_soloq_games, utility, "UTILITY", "l_utility_group", False)

In [79]:
w_top_soloq = w_top_soloq[["game_id", "win", "teamId", "teamPosition", "w_top_group"]]
l_top_soloq = l_top_soloq[["game_id", "win", "teamId", "teamPosition", "l_top_group"]]

w_jungle_soloq = w_jungle_soloq[["game_id", "win", "teamId", "teamPosition", "w_jungle_group"]]
l_jungle_soloq = l_jungle_soloq[["game_id", "win", "teamId", "teamPosition", "l_jungle_group"]]

w_mid_soloq = w_mid_soloq[["game_id", "win", "teamId", "teamPosition", "w_mid_group"]]
l_mid_soloq = l_mid_soloq[["game_id", "win", "teamId", "teamPosition", "l_mid_group"]]

w_bottom_soloq = w_bottom_soloq[["game_id", "win", "teamId", "teamPosition", "w_bottom_group"]]
l_bottom_soloq = l_bottom_soloq[["game_id", "win", "teamId", "teamPosition", "l_bottom_group"]]

w_utility_soloq = w_utility_soloq[["game_id", "win", "teamId", "teamPosition", "w_utility_group"]]
l_utility_soloq = l_utility_soloq[["game_id", "win", "teamId", "teamPosition", "l_utility_group"]]

In [94]:
from functools import reduce

top_soloq = pd.merge(left=w_top_soloq, right=l_top_soloq, on="game_id")[["game_id", "teamId_x", "w_top_group", "l_top_group"]]
jungle_soloq = pd.merge(left=w_jungle_soloq, right=l_jungle_soloq, left_on="game_id", right_on="game_id")[["game_id", "w_jungle_group", "l_jungle_group"]]
mid_soloq = pd.merge(left=w_mid_soloq, right=l_mid_soloq, left_on="game_id", right_on="game_id")[["game_id", "w_mid_group", "l_mid_group"]]
bottom_soloq = pd.merge(left=w_bottom_soloq, right=l_bottom_soloq, left_on="game_id", right_on="game_id")[["game_id", "w_bottom_group", "l_bottom_group"]]
utility_soloq = pd.merge(left=w_utility_soloq, right=l_utility_soloq, left_on="game_id", right_on="game_id")[["game_id", "w_utility_group", "l_utility_group"]]

dataframes = [top_soloq, jungle_soloq, mid_soloq, bottom_soloq, utility_soloq]
grouped_soloq = reduce(lambda left, right: pd.merge(left, right, on="game_id"), dataframes)

grouped_soloq = grouped_soloq.drop_duplicates("game_id")
grouped_soloq['winner_side'] = grouped_soloq.apply(lambda row: 100 if row['teamId_x'] == "Blue" else 200, axis=1)
grouped_soloq = grouped_soloq.drop("teamId_x", axis=1)
grouped_soloq = grouped_soloq[["game_id", "winner_side", 
                               "w_top_group", "l_top_group",
                               "w_jungle_group", "l_jungle_group",
                               "w_mid_group", "l_mid_group",
                               "w_bottom_group", "l_bottom_group",
                               "w_utility_group", "l_utility_group",]]
grouped_soloq

,game_id,winner_side,w_top_group,l_top_group,w_jungle_group,l_jungle_group,w_mid_group,l_mid_group,w_bottom_group,l_bottom_group,w_utility_group,l_utility_group
0,EUW1_5790486081,100,-1,-1,-1,-1,2,2,2,3,1,1
1,EUW1_5681053033,200,-1,0,-1,-1,1,0,3,0,1,1
2,EUW1_5749197239,200,-1,0,-1,-1,0,2,2,3,1,1
3,EUW1_5787496494,100,1,0,1,-1,0,0,2,3,1,0
4,EUW1_5744518531,100,0,0,-1,-1,0,0,3,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10793,KR_5829951103,200,-1,-1,-1,-1,0,0,0,0,1,0
10794,KR_5796884748,200,0,1,-1,-1,0,0,3,0,0,-1
10795,KR_5782484871,200,-1,1,-1,-1,0,0,1,0,0,0
10796,KR_5822322123,200,1,1,-1,-1,0,2,0,2,0,1


In [95]:
grouped_soloq.to_excel("../data/soloq/games_with_cluster_by_lane.xlsx")